## Import Packages

In [29]:
# Import necessary packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Helper Functions

In [30]:
def grayscale(img):
    # Convert mpimg color space into 1-channel grayscale
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

In [31]:
def gaussian_blur(img, kernel_size=5):
    # Apply Gaussian blur with default kernel size
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [32]:
def canny(img, low_threshold, high_threshold):
    # Canny edge detection
    return cv2.Canny(img, low_threshold, high_threshold)

In [33]:
def polygon_roi(img):
    if len(img.shape) == 3:
        r, c, _ = img.shape
        color_mask = (255, 255, 255)
    else:
        r, c = img.shape
        color_mask = 255
    offset_lb = c // 10
    offset_rb = c // 20
    offset_lt = c // 20
    offset_rt = c // 20
    offset_bottom = 0
    offset_top = r // 11
    vertices = np.array([[
        (0 + offset_lb, r + offset_bottom),           # bottom left
        (c // 2 - offset_lt, r // 2 + offset_top),    # top left
        (c // 2 + offset_rt, r // 2 + offset_top),    # top right
        (c - offset_rb, r + offset_bottom),           # bottom right
    ]], dtype=np.int32)
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, color_mask)
    masked = cv2.bitwise_and(img, mask)
    return masked

In [34]:
def hough_lines(img):
    # Distance resolution of the Hough grid (in pixels)
    rho = 2
    # Angular resolution of the Hough grid (in radians)
    theta = np.pi / 180
    # Min. # of votes required for a line
    threshold = 5
    # Min. # of pixels required for a line
    min_line_len = 20
    # Maximum gap between connectable line segments
    max_line_gap = 40
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]),
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

## Lane Line Detection

A 5-stage pipeline:
1. Convert to grayscale image
1. Apply Gaussian Blur
1. Detect Canny edges
1. Masking for ROI extraction
1. Detect Hough lines

In [35]:
def detect_lines(img):
    # Obtain grayscale image
    gray = grayscale(img)
    # Apply Gaussian blur filter
    blurred = gaussian_blur(gray, 3)
    # Detect Canny edges
    edges = canny(blurred, 50, 200)
    # Mask region of interest
    roi = polygon_roi(edges)
    # Detect Hough lines
    lines = hough_lines(roi)
    return lines

## Select candidates for lane lines

Given a list of lines, select among them those with
* both the endpoints within the x range of [`min_x`, `max_x`], and
* angle to the X-axis within the range [`min_angle`, `max_angle`].
This function is used for collecting line segments that are possibly left and right lane lines on the road.

In [36]:
def select_lines(lines, min_angle, max_angle, min_x, max_x):
    # Calculate tangent for each angle
    min_gradient = np.tan(np.radians(min_angle))
    max_gradient = np.tan(np.radians(max_angle))
    selected = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            # First we check the position of the line
            if min(x1, x2) < min_x or max(x1, x2) > max_x:
                continue
            # Vertical lines are not likely to be lane lines
            if x1 == x2:
                continue
            # We subtract y2 from y1 since y-axis is upside down
            m = (y1 - y2) / (x2 - x1)
            # For those selected lines, keep the following:
            # (1) gradient, (2) y-intercept, (3) segnemtn length,
            if min_gradient <= m <= max_gradient:
                selected.append(line)
    return selected

## Aggregate Multiple (similarly oriented) Lines

1. Calculate parameters of each line segment (gradient and y-intercept)
1. Calculate weight of each segment (based on its length)
1. Weighted averages for gradient and y-intercept --> this is the aggregated line
1. Provide endpoints, with one around the top (minimum y value) and the other around the bottom (a large value used so that the line extends across the lower border of the image)

In [43]:
def aggregate_lines(lines, max_y, min_y):
    gradients, intercepts, weights = [], [], []
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:
                continue
            m = (y2 - y1) / (x2 - x1)
            b = y1 - m * x1
            # Use each line's length as its weight in averaging
            l = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
            gradients.append(m)
            intercepts.append(b)
            weights.append(l)
    # Now we compute weighted (based on line length) average of the
    # gradient and y-intercept
    g, i, w = np.array(gradients), np.array(intercepts), np.array(weights)
    avg_m = np.average(g, weights=w)
    avg_b = np.average(i, weights=w)
    # Calculate low & high endpoints
    x1 = int((max_y - avg_b) / avg_m)
    x2 = int((min_y - avg_b) / avg_m)
    return x1, max_y, x2, min_y

## Drawing Functions

In [44]:
def draw_lines(img, lines, c=(255, 0, 0), t=2):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), c, t)
    return img

In [45]:
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Main Processor

Given an image, detect lane lines in it and annotate the left lane line with red and the right one with blue; return the annotated image.

In [49]:
def process_image(img):
    r = img.shape[0]
    c = img.shape[1]
    lines = detect_lines(img)
    # print("# of found lines: ", len(lines))
    left_candidates = select_lines(lines, 30, 45, 0, c // 2)
    # print("# of left lane line candidates: ", len(left_candidates))
    right_candidates = select_lines(lines, -45, -30, c // 2, c)
    # print("# of right lane line candidates: ", len(right_candidates))

    line_img = np.zeros_like(img)
    # line_img = draw_lines(line_img, left_candidates, c=(0, 255, 0))
    # line_img = draw_lines(line_img, right_candidates, c=(255, 255, 0))
    if len(left_candidates) > 0:
        left_lane_line = aggregate_lines(left_candidates, r, 3 * r // 5)
        line_img = draw_lines(line_img, [(left_lane_line, )], c=(255, 0, 0), t=10)
    if len(right_candidates) > 0:
        right_lane_line = aggregate_lines(right_candidates, r, 3 * r // 5)
        line_img = draw_lines(line_img, [(right_lane_line, )], c=(0, 0, 255), t=10)
    annotated = weighted_img(line_img, img)
    return annotated

## Tests on Still Image Files

In [50]:
import os

filenames = os.listdir("test_images")
for filename in filenames:
    pathname = os.path.join(os.getcwd(), "test_images", filename)
    # Read image file
    image = mpimg.imread(pathname)
    lane_line_detection = process_image(image)
    cv2_image = cv2.cvtColor(lane_line_detection, cv2.COLOR_RGB2BGR)
    output_pathname = os.path.join(os.getcwd(), "test_images_output", filename)
    cv2.imwrite(output_pathname, cv2_image)

## Tests on Sample Videos

In [51]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

filenames = os.listdir("test_videos")
for filename in filenames:
    pathname = os.path.join(os.getcwd(), "test_videos", filename)
    output_pathname = os.path.join(os.getcwd(), "test_videos_output", filename)
    clip = VideoFileClip(pathname)
    output_clip = clip.fl_image(process_image)
    %time output_clip.write_videofile(output_pathname, audio=False)

[MoviePy] >>>> Building video /src/test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video /src/test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:06<00:00, 24.62it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /src/test_videos_output/solidWhiteRight.mp4 

CPU times: user 2.31 s, sys: 304 ms, total: 2.61 s
Wall time: 7.27 s
[MoviePy] >>>> Building video /src/test_videos_output/challenge.mp4
[MoviePy] Writing video /src/test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:17<00:00, 15.50it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /src/test_videos_output/challenge.mp4 

CPU times: user 6.15 s, sys: 804 ms, total: 6.95 s
Wall time: 19.5 s
[MoviePy] >>>> Building video /src/test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video /src/test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:21<00:00, 32.03it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /src/test_videos_output/solidYellowLeft.mp4 

CPU times: user 7.51 s, sys: 893 ms, total: 8.4 s
Wall time: 22.2 s
